In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Texto de ejemplo
corpus = """
La margarita florece con la llegada de la primavera
La margarita baila al ritmo suave del viento
La margarita es testigo del paso del tiempo en el jardín
La margarita despierta con el primer rayo de sol
La margarita guarda secretos entre sus pétalos blancos
La margarita es una joya natural en el prado
La margarita saluda a cada visita con gracia
La margarita brilla como una estrella en el campo
La margarita se inclina ante la lluvia, agradecida
La margarita susurra historias al viento
La margarita es la musa de los poetas y soñadores
La margarita tiñe el paisaje con su blancura radiante
La margarita anuncia la llegada de la estación cálida
La margarita se mantiene firme ante las tormentas
La margarita comparte su fragancia con el aire
La margarita guarda la esencia de la naturaleza en sus pétalos
La margarita refleja la pureza del alma en su simplicidad
La margarita adorna el prado como un regalo divino
La margarita invita a las abejas a danzar a su alrededor
La margarita es un faro de luz en el campo verde
La margarita resiste con gracia los caprichos del clima
La margarita saluda al sol con su rostro radiante
La margarita se abre paso entre la hierba alta
La margarita revela sus secretos al amanecer
La margarita es la reina silenciosa del jardín
La margarita es un símbolo de esperanza y renovación
La margarita se balancea con la melodía de la naturaleza
La margarita es un recordatorio de la belleza simple
La margarita comparte su amor con el mundo
La margarita observa con calma el vaivén de las estaciones
La margarita es un pequeño faro de luz en medio del prado
La margarita parece susurrar secretos al viento que solo la naturaleza comprende
La margarita, con sus pétalos blancos, es como un lienzo en blanco pintado por la naturaleza
La margarita despliega su belleza con la simplicidad que solo la naturaleza puede lograr
La margarita es testigo silencioso de la danza de las abejas en su búsqueda de néctar
La margarita, con su frescura matutina, saluda al sol con gracia y elegancia
La margarita es un recordatorio de que la belleza a menudo se encuentra en la sencillez
La margarita parece capturar la esencia misma de la serenidad en cada uno de sus pétalos
La margarita, como una obra maestra efímera, despierta admiración en su breve existencia
La margarita revela sus secretos solo a aquellos que se toman el tiempo de observarla con detenimiento
La margarita en el jardín despliega sus pétalos al sol de la mañana
La margarita blanca destaca entre el verde vibrante del prado
La margarita me recuerda a los días de verano y la brisa suave
La margarita es un símbolo de pureza y belleza natural
La margarita se balancea con gracia en el viento primaveral
La margarita, modesta pero hermosa, adorna el borde del camino
La margarita guarda secretos de amores infantiles y susurros de la naturaleza
La margarita revela su esplendor al abrirse con delicadeza al nuevo día
La margarita es una joya efímera que adorna el campo con su presencia
La margarita, pequeña y valiente, desafía los elementos con su fragilidad
"""

# Preprocesamiento del texto
tokens = set(corpus.split())
word_to_idx = {word: idx for idx, word in enumerate(tokens)}
idx_to_word = {idx: word for idx, word in enumerate(tokens)}
num_tokens = len(tokens)

# Crear pares de entrada y objetivo
input_sequence = [word_to_idx[word] for word in corpus.split()]
target_sequence = input_sequence[1:] + [word_to_idx.get(".", 0)]

# Convertir a tensores de PyTorch
input_sequence = torch.LongTensor(input_sequence)
target_sequence = torch.LongTensor(target_sequence)

class TextGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout_rate):
        super(TextGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # Convierte las palabras (codigificadas como números) en una secuencia (vector) de un tamaño dado
        # Esto es lo que va a recordar secuencias de palabras que van bien juntas (que aparecen en nuestro corpus)
        #
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout_rate) # Esta es la capa que recuerda los datos que ya hemos procesado.
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)  # Descartar aleatoriamente algunas neuronas en cada etapa del proceso de aprendizaje para que demos oportunidad a otras neuronas a aprender más
        self.fc2 = nn.Linear(hidden_size, vocab_size) # Capa lineal de neuronas, que tiene tantas neuronas como token en nuestro corpus

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        x = self.relu(self.fc1(lstm_out))
        x = self.dropout(x)
        output = self.fc2(x)
        return output


# Configuración de hiperparámetros actualizada
embedding_dim = 150
hidden_size = 500
learning_rate = 0.01
num_epochs = 150
num_layers = 3  # Son las cpas de memoria que conectamos : LSTM
dropout_rate = 0.2  # Ajusta según sea necesario


# Inicializar el modelo, la función de pérdida y el optimizador
model = TextGenerator(num_tokens, embedding_dim, hidden_size, num_layers, dropout_rate)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Entrenamiento del modelo con los nuevos hiperparámetros
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(input_sequence.unsqueeze(0))
    loss = criterion(output.view(-1, num_tokens), target_sequence)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# seed_text Es el texto del que partimos a la hora de generar un nuevo texto
# length Longitud del texto que vamos a generar
# Temperature: Lo creativo o poco creativo que va a ser el texto que vamos a generar
#     Valores muy bajos-> 0, implican que seremos poco creativos
#     Valores más altos implican que seremos muy creativos
def generate_text(model, seed_text, length=50, temperature=1.0):
    model.eval()
    with torch.no_grad():
        seed_sequence = [word_to_idx[word] for word in seed_text.split()]
        generated_sequence = seed_sequence.copy()

        for _ in range(length):
            input_tensor = torch.LongTensor(seed_sequence).unsqueeze(0)
            output_probs = model(input_tensor) # Obteniendo las probabilidades de que cada palabra del corpus encaje detrás de ese texto del que partimos
            output_probs = output_probs[:, -1, :] / temperature  # 1
            # 1 lo deja como está (la probabilidad)
            # 0 aumenta la probabilidad
            # Número mayor que 1 me disminuye la probabilidad

            # De todas las palabras del corpus me quedo con 1, en base a esas probabilidades... que he trucado con la TEMPERATURA
            softmax_probs = nn.functional.softmax(output_probs, dim=-1)
            predicted_idx = torch.multinomial(softmax_probs, 1).item()

            # Añadimos la nueva palabra a la secuencia que llevo... y empezamos de nuevo... así como tantas palabras que me digan.
            generated_sequence.append(predicted_idx)
            seed_sequence = generated_sequence[-2:]


        # Hemos acabado generando una secuencia nueva de Números, que nos toca convertir ahora en palabras
        generated_text = ' '.join([idx_to_word[idx] for idx in generated_sequence])
        return generated_text

# Generar texto de prueba
seed_text = "La margarita"
generated_text = generate_text(model, seed_text, length=8, temperature=2)
print("Texto generado:", generated_text)


Epoch [10/150], Loss: 4.5264
Epoch [20/150], Loss: 4.4211
Epoch [30/150], Loss: 4.4302
Epoch [40/150], Loss: 4.4127
Epoch [50/150], Loss: 4.4152
Epoch [60/150], Loss: 4.4186
Epoch [70/150], Loss: 4.4093
Epoch [80/150], Loss: 4.4203
Epoch [90/150], Loss: 4.4117
Epoch [100/150], Loss: 4.4184
Epoch [110/150], Loss: 4.4286
Epoch [120/150], Loss: 4.4367
Epoch [130/150], Loss: 4.4057
Epoch [140/150], Loss: 4.4299
Epoch [150/150], Loss: 4.4219
Texto generado: La margarita su es existencia margarita hermosa, campo la comparte


In [13]:
seed_text = "La margarita florece"
generated_text = generate_text(model, seed_text, length=20, temperature=1)
print("Texto generado:", generated_text)


Texto generado: La margarita florece blancos, solo recuerda firme camino La tiñe ante un delicadeza tiempo la La esperanza un margarita, sol me la susurros
